In [ ]:
import sys
sys.path.insert(1, '/path/to/paste2/src')
from paste2 import PASTE2, model_selection, projection
import anndata
import os
import os.path as osp
import scanpy as sc

In [ ]:
# you can download the data from https://drive.google.com/file/d/1NMvNjcWGtD4naNNmwe0mEcfnZTSitwO9/view?usp=sharing
data_dir = "path/to/data" # /mnt/spatial_datasets
data_type = "10x"
dataset_dir = 'human_dorsolateral_pre-frontal_cortex'
sample_ids = ["151673", "151674", "151675", "151676"]

In [ ]:
def read_slices(sample_ids):
    adatas = []
    for i, sample_id in enumerate(sample_ids):
        adatas.append(sc.read_visium(osp.join(data_dir, dataset_dir, sample_id), count_file="{}_filtered_feature_bc_matrix.h5".format(sample_id)))
        adatas[i].var_names_make_unique()
        adatas[i].obs.index = sample_id + "_" + adatas[i].obs.index
    return adatas

In [ ]:
def combine_slices_every_k(sample_ids, k=4):
    adatas = read_slices(sample_ids)
    assert len(sample_ids) % k == 0
    num_batch = len(sample_ids) // k
    for i in range(num_batch):
        slices = adatas[i*k:(i+1)*k]
        pis = []
        for j in range(k-1):
            sliceA = slices[j]
            sliceB = slices[j+1]
            estimated_s = model_selection.select_overlap_fraction(sliceA, sliceB)
            pis.append(PASTE2.partial_pairwise_align(sliceA, sliceB, s=estimated_s))
        new_slices = projection.partial_stack_slices_pairwise(slices, pis)
        combined_adata = anndata.concat(new_slices, join="outer", label="batch")
        combined_adata.var = slices[0].var
        os.makedirs(osp.join(data_dir, dataset_dir, "_".join(sample_ids[i*k:(i+1)*k])), exist_ok=True)
        combined_adata.write_h5ad(osp.join(data_dir, dataset_dir, "_".join(sample_ids[i*k:(i+1)*k]), "_".join(sample_ids[i*k:(i+1)*k]) + ".h5ad"), compression="gzip")

In [ ]:
combine_slices_every_k(sample_ids, k=4)